In [ ]:
import os 
from google.colab import drive

In [ ]:
!pip install tsmoothie

In [ ]:
import numpy as np
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import tsmoothie
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Conv1D,Input, MaxPooling1D
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from sklearn.decomposition import PCA
from tensorflow.keras import optimizers
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline

In [ ]:
#LSTM=tf.compat.v1.keras.layers.CuDNNLSTM 
LSTM = tf.compat.v1.keras.layers.LSTM
GRU = tf.compat.v1.keras.layers.GRU
#GRU = tf.compat.v1.keras.layers.CuDNNGRU
a=tf.compat.v1.keras.layers.CuDNNLSTM
smoother=tsmoothie.KalmanSmoother(component='level_trend',  component_noise={'level':0.1, 'trend':0.1})

In [ ]:
data_path = "CNN_TLSTM dataset.csv"

In [ ]:
# loading data

data_1=pd.read_csv(data_path)
print(data_1.head())
print(data_1.columns)
data=data_1[['Open','High',
             'Low','Close','Nasdaq Index (NI)',
             'Dow-Jones  Industrial \nAverage (DJIA)',
             'Shanghai Composite Index (SCI)',
             'Hang Seng Index (HSI)']]
dates=data_1[['Date']]
#Interpolation
data[['Open']] = data[['Open']].interpolate()
data[['High']] = data[['High']].interpolate()
data[['Low']] = data[['Low']].interpolate()
data[['Close']] = data[['Close']].interpolate()
data[['Nasdaq Index (NI)']] = data[['Nasdaq Index (NI)']].interpolate()
data[['Dow-Jones  Industrial \nAverage (DJIA)']] = data[['Dow-Jones  Industrial \nAverage (DJIA)']].interpolate()
data[['Shanghai Composite Index (SCI)']] = data[['Shanghai Composite Index (SCI)']].interpolate()
data[['Hang Seng Index (HSI)']] = data[['Hang Seng Index (HSI)']].interpolate()

# normalising data

Open=MinMaxScaler()
High=MinMaxScaler()
Low=MinMaxScaler()
Close=MinMaxScaler()
Volume=MinMaxScaler()
Lowest=MinMaxScaler()

dataaa=np.zeros((data.shape),dtype=np.float64)

a=Open.fit_transform(data[['Open']])
dataaa[:,0] = a[:,0]
a=High.fit_transform(data[['High']])
dataaa[:,1] = a[:,0]
a=Low.fit_transform(data[['Low']])
dataaa[:,2] = a[:,0]
a=Close.fit_transform(data[['Close']])
dataaa[:,3] = a[:,0]
a=Volume.fit_transform(data[['Nasdaq Index (NI)']])
dataaa[:,4] = a[:,0]
a=Lowest.fit_transform(data[['Dow-Jones  Industrial \nAverage (DJIA)']])
dataaa[:,5] = a[:,0]
a=Open.fit_transform(data[['Shanghai Composite Index (SCI)']])
dataaa[:,6] = a[:,0]
a=Open.fit_transform(data[['Hang Seng Index (HSI)']])

a=dataaa[:,0]
smoother.smooth(a)
dataaa[:,0]=smoother.smooth_data

a=dataaa[:,1]
smoother.smooth(a)
dataaa[:,1]=smoother.smooth_data

a=dataaa[:,2]
smoother.smooth(a)
dataaa[:,2]=smoother.smooth_data

a=dataaa[:,3]
smoother.smooth(a)
dataaa[:,3]=smoother.smooth_data

a=dataaa[:,4]
smoother.smooth(a)
dataaa[:,4]=smoother.smooth_data

a=dataaa[:,5]
smoother.smooth(a)
dataaa[:,5]=smoother.smooth_data

a=dataaa[:,6]
smoother.smooth(a)
dataaa[:,6]=smoother.smooth_data


full_data=dataaa

# finding total number of days

data=dataaa


total_days=data.shape[0]
print(total_days)

         Date      Open  ...  Shanghai Composite Index (SCI)  Hang Seng Index (HSI)
0  2003-12-01  1.203398  ...                          1432.7                12457.0
1  2003-12-02  1.196101  ...                          1435.3                12412.2
2  2003-12-03  1.209000  ...                          1445.0                12361.2
3  2003-12-04  1.212004  ...                          1452.8                12342.7
4  2003-12-05  1.207802  ...                          1451.1                12314.7

[5 rows x 9 columns]
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Nasdaq Index (NI)',
       'Dow-Jones  Industrial \nAverage (DJIA)',
       'Shanghai Composite Index (SCI)', 'Hang Seng Index (HSI)'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


4652


In [ ]:
training_period_dates=dates.loc[:int(full_data.shape[0]*1)]
#testing_period_dates=dates.loc[int(full_data.shape[0]*0.9):]

In [ ]:
total_days=data.shape[0]

timesteps=10

Train_data=data[:round(total_days*1),:]
#Test_data=data[round(total_days*0.9):,:]
print(Train_data.shape[0]," Days are used for Training")
#print(Test_data.shape[0]," Days are used for Testing")
# Function for ordering or making sequence of our test and train data according to date
def make_sequence(data,timesteps):
    X_train=[]
    Y_train=[]
    for i in range(len(data)-timesteps):
        X_train.append(data[i:(i+timesteps),[0,1,2,4,5,6,7]])
        Y_train.append(data[i+timesteps,3])
    return X_train,Y_train
Train_data,Train_Label=make_sequence(Train_data,timesteps)
Train_data,Train_Label=np.array(Train_data),np.array(Train_Label)
#Test_data,Test_Label=make_sequence(Test_data,timesteps)
#Test_data,Test_Label=np.array(Test_data),np.array(Test_Label)
input_shape=((Train_data).shape[1],(Train_data).shape[2])
print(input_shape)
Train_Label = Train_Label.reshape(-1,1)


4652  Days are used for Training
(10, 7)


In [ ]:
#model building
model=Sequential()
print(input_shape)
#tf.compat.v1.disable_eager_execution()
#adding layers to the model
x = Input(shape=input_shape)
model.add(Conv1D(32,1,input_shape=input_shape, activation="sigmoid"))
model.add(MaxPooling1D(pool_size=1,strides=1))
model.add(LSTM(64))

y=Dense(1)(x)
model.add(Dense(1))

print(model.summary())

(10, 7)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 10, 32)            256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 10, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 25,153
Trainable params: 25,153
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#compiling model
lr=0.001
mae = tf.keras.losses.MeanAbsoluteError()
model.compile(optimizer = optimizers.Adam(learning_rate= lr), loss = mae)
#fitting model
history=model.fit(Train_data,Train_Label,batch_size=60,epochs=50)

Epoch 1/50
78/78 [==============================] - 4s 17ms/step - loss: 0.1484
Epoch 2/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0372
Epoch 3/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0298
Epoch 4/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0209
Epoch 5/50
78/78 [==============================] - 1s 16ms/step - loss: 0.0219
Epoch 6/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0204
Epoch 7/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0183
Epoch 8/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0205
Epoch 9/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0183
Epoch 10/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0171
Epoch 11/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0171
Epoch 12/50
78/78 [==============================] - 1s 17ms/step - loss: 0.0144
Epoch 13/50
78/78 [==================

In [ ]:
#predicting values from our model of test data
Test_Predicted_Label=model.predict(Train_data)
print(Test_Predicted_Label.shape)
print(Test_Predicted_Label)
Test_Label = Train_Label.reshape(Train_Label.shape[0],1)
y_hat_test = Close.inverse_transform(Test_Predicted_Label)
#y_test = Close.inverse_transform(Test_Label)
np.savetxt("predicted.csv", y_hat_test, delimiter=",")

(4642, 1)
[[0.33036584]
 [0.33557677]
 [0.34252748]
 ...
 [0.22993915]
 [0.22963408]
 [0.22852671]]
